In [20]:
import numpy as np
import re
import time

In [21]:
bool(re.match('[a-z]','a'))

True

In [22]:
filename = "input"
file = open(filename + ".txt")

seeds = np.array(file.readline().removeprefix('seeds:').split(),dtype=int)
file.readline()
maps = []

map = []

for line in file.readlines():
    if line == '\n':
        npmap = np.array(map,dtype=int)
        slicer = np.argsort(npmap[:,1])
        maps.append(npmap[slicer])
    elif bool(re.match('[a-z]',line[0])):
        map = []
    else:
        map.append(line.split())
npmap = np.array(map,dtype=int)
slicer = np.argsort(npmap[:,1])
maps.append(npmap[slicer])
def readmap(maps, source):
    index = maps[:,1] <= source
    if sum(index)==0:
        return source
    i = np.argmax(maps[:,1][index])
    map = maps[i]
    if (map[1]+map[2] <= source):
        return source
    else:
        return map[0] + source - map[1] 

minval = np.inf
for seed in seeds:
    val = seed
    # print(val,end='')
    for map in maps:
        val = readmap(map, val)
        # print('-'+str(val),end='')
    if val < minval:
        minval = val
minval

214922730

In [28]:
filename = "input"
file = open(filename + ".txt")

start_time = time.time()

seeds = np.array(file.readline().removeprefix('seeds:').split(),dtype=int)
seedranges = np.array([[seeds[2*i],seeds[2*i] + seeds[2*i+1]-1] for i in range(int(len(seeds)/2))])

file.readline()
maps = []

map = []

for line in file.readlines():
    if line == '\n':
        npmap = np.array(map,dtype=int)
        npmap[:,2] = npmap[:,1] + npmap[:,2] -1
        slicer = np.argsort(npmap[:,1])
        maps.append(npmap[slicer])
    elif bool(re.match('[a-z]',line[0])):
        map = []
    else:
        map.append(line.split())

npmap = np.array(map,dtype=int)
npmap[:,2] = npmap[:,1] + npmap[:,2] -1
slicer = np.argsort(npmap[:,1])
maps.append(npmap[slicer])

minseed = min(seedranges[:,0])
maxseed = max(seedranges[:,1])

datarange = [minseed,maxseed]

seedranges = [(r[0],r[1]) for r in seedranges]

result = {}

def readpaths(maps, inp_range):
    paths = {}
    current = inp_range[0]
    maxinp = max(maps[:,2])

    if inp_range[1] <= maps[0,1] or inp_range[0] > maps[-1,2]: # full underflow or full overflow
        missedflow = (inp_range[0],inp_range[1])
        return {missedflow : missedflow}
    
    if inp_range[0] < maps[0,1]: #partial underflow
        underflow = (inp_range[0],maps[0,1]-1)
        paths[underflow] = underflow
        current = maps[0,1]

    while current < min(inp_range[1], maxinp):
        index = maps[:,1] <= current
        next = int(min(max(maps[index,2]), inp_range[1]))
        key = (current, next)
        val = (readmap(maps, current), readmap(maps, next))
        paths[key] = val        
        current = next+1
        

    if current <= inp_range[1]:
        overflow = (current, inp_range[1])
        paths[overflow] = overflow


    return paths

def split_paths(maps, input):
    result = []
    current = input[0]
    maxinp = max(maps[:,2])

    if input[1] <= maps[0,1] or input[0] > maps[-1,2]: # full underflow or full overflow
        missedflow = (input[0],input[1])
        return [missedflow]
    
    if input[0] < maps[0,1]: #partial underflow
        underflow = (input[0],maps[0,1]-1)
        result.append(underflow)
        current = maps[0,1]

    while current < min(input[1], maxinp):
        index = maps[:,1] <= current
        I = sum(index)-1
        if current <= max(maps[index,2]):
            next = int(min(max(maps[index,2]), input[1]))
            key = (current, next)
            val = (readmap(maps, current), readmap(maps, next))
            result.append(val)     
            current = next+1
        else: #You are in an intermediate area
            next = int(maps[I+1,1])-1
            result.append((current, next))
            current = next+1
        

        

    if current <= input[1]:
        overflow = (current, input[1])
        result.append(overflow)

    
    return result

def left_sort(input):
    index = np.argsort([i[0] for i in input])
    return [input[i] for i in index]
                
def merge_ranges(input):
    new_list = []
    new_list.append(input[0])
    for subrange in input[1::]:
        if subrange[0] <= new_list[-1][1]+1:
            new_list[-1] = (new_list[-1][0],max(subrange[1],new_list[-1][1]))
        else:
            new_list.append(subrange)
    return new_list

ranges = [seedrange for seedrange in seedranges]
for map in maps:
    newranges = []
    for subrange in ranges:
        split_ranges = split_paths(map, subrange)
        for newrange in split_ranges:
            newranges.append(newrange)
    ranges = newranges
    ranges = left_sort(ranges)
    ranges = merge_ranges(ranges)

minval = np.inf
for subrange in ranges:
    minval = subrange[0] if subrange[0] < minval else minval
minval

end_time = time.time()
print(f'{minval}, time: {end_time-start_time}')

148041808, time: 0.03235936164855957


In [25]:
print(f'{minval}, time: {end_time-start_time}')

148041808, time: 0.043244123458862305
